In [77]:
import pandas as pd
import matplotlib.pyplot as plt

#csv 파일 읽기
df = pd.read_csv("Covid Data.csv")
df['died'] = df['DATE_DIED'] != '9999-99-99'

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 22 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

In [79]:
# USER = 1차 진료 기관 여부 1 = YES 2= NO
#medical_unit	진료받은 의료 기관 코드	병원 종류나 지역
#sex	성별	1=남자, 2=여자
#patient_type	외래 vs 입원 여부	1=외래, 2=입원
#date_died	사망일	'9999-99-99'이면 아직 생존
#intubed	삽관 여부	1=Yes, 2=No, 97=알 수 없음
#pneumonia	폐렴 여부	1=Yes, 2=No
#age	나이	숫자
#pregnant	임신 여부 (여성만 해당)	1=Yes, 2=No
#diabetes	당뇨 여부	1=Yes, 2=No
#copd	만성폐쇄성폐질환 여부	1=Yes, 2=No
#asthma	천식 여부	1=Yes, 2=No
#inmsupr	면역억제 여부	1=Yes, 2=No
#hipertension	고혈압 여부	1=Yes, 2=No
#other_disease	기타 질환	1=Yes, 2=No
# cardiovascular	심혈관 질환	1=Yes, 2=No
#obesity	비만 여부	1=Yes, 2=No
#renal_chronic	만성 신장 질환	1=Yes, 2=No
#tobacco	흡연 여부	1=Yes, 2=No
#clasiffication_final	코로나 확진 여부	1=양성, 2=음성, 3=의심
#icu	중환자실 입실 여부	1=Yes, 2=No, 97=모름


In [80]:
# 전체 중 고혈압 환자
high_hpt=df[df['HIPERTENSION']==1]

#전체 중 고혈압 아닌 환자
no_hpt = df[df['HIPERTENSION']==2]

In [81]:
print(f"총 {len(high_hpt)}명 고혈압 환자 데이터")

총 162729명 고혈압 환자 데이터


In [82]:
hpt_count=round(len(hpt)/len(df)*100,3)
print(f"고혈압 환자 비율: {hpt_count}%")

고혈압 환자 비율: 15.519%


In [83]:
#고혈압 중 사망자
dead_high_hpt=df[(df['HIPERTENSION'] == 1) & (df['DATE_DIED'] != '9999-99-99')]
print(f"{len(dead_high_hpt)}명")

32061명


In [84]:
# 고혈압 없는 사람 중 사망자
dead_no_hpt=df[(df['HIPERTENSION'] == 2) & (df['DATE_DIED'] != '9999-99-99')]
print(f"{len(dead_no_hpt)}명")

44191명


In [85]:
death_rate_high = round(len(dead_high_hpt)/len(high_hpt)*100,3)
death_rate_no = round(len(dead_no_hpt)/len(no_hpt)*100,3)

In [86]:
print(f"고혈압 환자 사망률 : {death_rate_high}")
print(f"고혈압 없는 환자 사망률 : {death_rate_no}")

고혈압 환자 사망률 : 19.702
고혈압 없는 환자 사망률 : 5.006


코로나 확진자 분석

In [87]:
#기저 질환이 코로나 사망률에 끼치는 영향

In [88]:
# 코로나 확진자
cvd = df[df['CLASIFFICATION_FINAL'] ==1]

In [89]:
# 코로나 확진자 사망자
dead_cvd = cvd[cvd['DATE_DIED'] != '9999-99-99']

In [90]:
#기저 질환 리스트
disease_list = ['DIABETES','HIPERTENSION','OBESITY']
results = {}

In [93]:
for disease in disease_list :
    with_disease = cvd[cvd[disease]==1]
    without_disease = cvd[cvd[disease]==2]
    
    death_rate_with = round(with_disease['died'].mean()*100,3)
    death_rate_without = round(without_disease['died'].mean()*100,3)
    
    results[disease] = {
        'with': death_rate_with,
        'without': death_rate_without
    }

for disease, rates in results.items():
    print(f"{disease.capitalize()}:")
    print(f"  * With    : {rates['with']}%")
    print(f"  * Without : {rates['without']}%\n")


Diabetes:
  * With    : 30.885%
  * Without : 7.695%

Hipertension:
  * With    : 25.768%
  * Without : 7.942%

Obesity:
  * With    : 14.687%
  * Without : 10.142%

